In [ ]:
from sklearn.datasets import load_svmlight_file
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd

libsvm_file1311 = 'Ratings_final_output.libsvm'
sparse_matrix1311, _ = load_svmlight_file(libsvm_file1311)

knn1311 = NearestNeighbors(metric='cosine', algorithm='brute')
knn1311.fit(sparse_matrix1311)

book_metadata = {i: f"Book {i}" for i in range(sparse_matrix1311.shape[1])}
def recommend_books_for_all_users1311(k=10, top_n=5):
    """Recommend top N books for all users and output results to a CSV."""
    all_recommendations = []
    
    for user_id1311 in range(sparse_matrix1311.shape[0]):
        # Step 1: Find the K most similar users
        distances1311, indices1311 = knn1311.kneighbors(sparse_matrix1311[user_id1311], n_neighbors=k + 1)
        similar_users1311 = indices1311.flatten()[1:]
        similarity_scores1311 = 1 - distances1311.flatten()[1:]       
        # Step 2: Identify books read by similar users
        similar_users_matrix1311 = sparse_matrix1311[similar_users1311].toarray()        
        # Step 3: Calculate weighted average ratings
        weighted_sum1311 = np.dot(similarity_scores1311, similar_users_matrix1311)
        similarity_sum1311 = np.sum(similarity_scores1311[:, None] * (similar_users_matrix1311 > 0), axis=0)
        weighted_avg_ratings1311 = np.divide(weighted_sum1311, similarity_sum1311,
                                            out=np.zeros_like(weighted_sum1311),
                                            where=similarity_sum1311 > 0)

        user_rated_books1311 = sparse_matrix1311[user_id1311].toarray().flatten() > 0
        weighted_avg_ratings1311[user_rated_books1311] = 0
        
        # Step 4: Get top N recommended books
        recommended_books1311 = np.argsort(-weighted_avg_ratings1311)[:top_n]

        for book_id in recommended_books1311:
            all_recommendations.append({
                "User_ID": user_id1311,
                "Book_ID": book_id,
                "Book_Title": book_metadata.get(book_id, f"Book {book_id}"),
                "Recommendation_Score": weighted_avg_ratings1311[book_id]
            })

    recommendations_df = pd.DataFrame(all_recommendations)
    recommendations_df.to_csv('all_user_recommendations.csv', index=False)
    print(f"Recommendations for all users saved to 'all_user_recommendations.csv'.")

# Generate recommendations for all users
recommend_books_for_all_users1311(k=10, top_n=5)